In [3]:
import cv2
import numpy as np
import os
import math

class ObjectDetection:
    def __init__(self, weights_path="dnn_model/yolov4-tiny.weights", cfg_path="dnn_model/yolov4-tiny.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608

        # Load Network
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.classes = []
        self.load_class_names()
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="dnn_model/classes.txt"):

        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        self.colors = np.random.uniform(0, 255, size=(80, 3))
        return self.classes

    def detect(self, frame):
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)



In [7]:
class DetectPedestrain():
    def __init__(self, inputPath, outputPath):
        self.inputPath = inputPath
        self.outputPath = outputPath
        self.fileList = []
        self.od =  ObjectDetection()
        
        # Get all images in a folder
        for file_name in os.listdir(inputPath):
            self.fileList.append(file_name)
        
    def showImg(self, img):
        cv2.imshow("image",img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()  
    
    # Detect all images in a Folder
    def detectImg(self):
        for imgFile in self.fileList:
            img = cv2.imread(self.inputPath + imgFile)
            
            class_ids, scores, boxes  = self.od.detect(img)
            
            for index, box in enumerate(boxes):
                if class_ids[index] == 0:
#                 if class_ids[index] != 0:
                    (x, y, w, h) = box
                    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.imwrite(self.outputPath + imgFile, img)
    
    # Detect all video in a folder
    def detectAllVideo(self):
        for v in self.fileList:
            print('FileName:', v)
            self.detectVideo(v)
        
    def detectVideo(self, video):
        cap = cv2.VideoCapture(self.inputPath + video)
        
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') 
        cap_fps = 10
        size = (1920,1080)
        outputVideo = cv2.VideoWriter(self.outputPath + video, fourcc, cap_fps, size)

        count = 0  # Frame number

        center_points_prev = []  # Center point of boxes in previous frame

        track_objects = {}  # Tracked objects

        track_id = 0  # Tracked ID from 0 
        
        while True:
            count += 1
            print('------------------------')
            print('Frame:', count)

            # Read each fram of video
            success, img = cap.read()

            # If video is over, break the loop
            if success == False:
                break
            center_points_current = []  # Store all center points of boxes in this frame
    
            class_ids, scores, boxes  = self.od.detect(img)

            # Draw box for each detected people
            for index, box in enumerate(boxes):
                if class_ids[index] == 0:
                    (x, y, w, h) = box
                    center_points_current.append((int((x+x+w)/2), int((y+y+h)/2) ))

                    # Draw rectangle
                    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            
            if count <= 2:
            # Process the first2 frames
                for pt1 in center_points_current:  #  Center points in current frame
                    for pt2 in center_points_prev:  # Center points in previous frame
                        # Count the distance of two center points
                        distance = math.hypot(pt2[0]-pt1[0], pt2[1]-pt1[1])

                        #  If distance is less than 20, we can define they are same object in 2 frames
                        if distance < 20:
                            # Record the tracked objects
                            track_objects[track_id] = pt1
                            track_id += 1


            # Process objects in frames after first 2
            else:
                # Copy first before loop
                track_objects_new = track_objects.copy()

                for object_id, pt2 in track_objects_new.items(): 
                    # Record if the objects in previous frame still exists in current frame
                    object_exist = False
                    for pt1 in center_points_current:  
                        distance = math.hypot(pt2[0]-pt1[0], pt2[1]-pt1[1])
                        if distance < 20:
                            # Update the center point position
                            track_objects[object_id] = pt1
                            object_exist = True
                            # Delete updated center points in current frame, the left are new objects
                            center_points_current.remove(pt1)
                            continue
                    # If the objects disappears, delete its id
                    if object_exist == False:
                        track_objects.pop(object_id)

                #（Process new objects
                for pt in center_points_current:
                    track_objects[track_id] = pt
                    track_id += 1

            # Draw in each frame
            for object_id, pt in track_objects.items():

                # Draw circle in center points
                cv2.circle(img, pt, 5, (255,0,0), -1)

                cv2.putText(img, str(object_id), (pt[0], pt[1]-5), 0, 1, (0,0,255),2)
    
#                 cTime = time.time() 
#                 fps = 1/(cTime-pTime)
#                 pTime = cTime  

#                 cv2.putText(img, str(int(fps)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)  

                # Copy the center points information of this frame for next frame
                center_points_prev = center_points_current.copy()
                
            outputVideo.write(img)
            if cv2.waitKey(110) & 0xFF==27:  
                break
            


In [16]:
# Detecct in video
inVideoPath = "Traffic Dataset/"
oVideoPath ="result/Videos/"

dVideo = DetectPedestrain(inVideoPath, oVideoPath)

dVideo.detectVideo("test_200f.mp4")

Loading Object Detection
Running opencv dnn with YOLOv4
------------------------
Frame: 1
------------------------
Frame: 2
------------------------
Frame: 3
------------------------
Frame: 4
------------------------
Frame: 5
------------------------
Frame: 6
------------------------
Frame: 7
------------------------
Frame: 8
------------------------
Frame: 9
------------------------
Frame: 10
------------------------
Frame: 11
------------------------
Frame: 12
------------------------
Frame: 13
------------------------
Frame: 14
------------------------
Frame: 15
------------------------
Frame: 16
------------------------
Frame: 17
------------------------
Frame: 18
------------------------
Frame: 19
------------------------
Frame: 20
------------------------
Frame: 21
------------------------
Frame: 22
------------------------
Frame: 23
------------------------
Frame: 24
------------------------
Frame: 25
------------------------
Frame: 26
------------------------
Frame: 27
--------

In [8]:
# Detect for image
inImagePath = "image/"
oImagePath = "result/images/"

dImage = DetectPedestrain(inImagePath, oImagePath)
dImage.detectImg()

Loading Object Detection
Running opencv dnn with YOLOv4
